In [1]:
!pip install ipywidgets

In [2]:
from ipywidgets import Video
from IPython.display import display
import os
import pathlib
import cv2
from tqdm.notebook import tqdm
import subprocess
import json
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as D
import numpy as np
import pickle
import random
import torch
import shutil


In [3]:
#수어 스크립트 시작<START> 인덱스
STA_INDEX = 1

#수어 스크립트 끝 <END> 인덱스
END_INDEX = 2

#동영상의 3의 배수 프레임 마다 프레임 저장
SKIP_NUM=3

In [4]:
#수어 동영상 keypoint 개수
MAX_FRAME_NUM = 240

In [5]:
MAX_FRAME_NUM

240

In [6]:
HID_DIM = 512

In [7]:
#index_to_word.pickle: Decoder가 예측한 숫자를 어떤 단어로 변환 해야 하는지 저장한 파일
with open('data/index_to_word.pickle','rb') as f:
    #Decoder가 예측한 숫자를 어떤 단어로 변환 해야 하는지 저장한 파일을 읽어서 index_to_word 에 저장
    index_to_word = pickle.load(f)

In [8]:
index_to_word

{0: '<PAD>',
 1: '<START>',
 2: '<END>',
 3: '6분',
 4: '깊고',
 5: '고흥군',
 6: '되었을',
 7: '빼고',
 8: '20분',
 9: '위경련',
 10: '뒤집',
 11: '벗고',
 12: '털어',
 13: '지탱',
 14: '휴식',
 15: '우리',
 16: '용문산',
 17: '35',
 18: '1시',
 19: '된',
 20: '14',
 21: '영등포구',
 22: '바닥',
 23: '지키시기',
 24: '되었습니다',
 25: '농사일',
 26: '도크',
 27: '식수',
 28: '별',
 29: '위험하므로',
 30: '건물',
 31: '세종',
 32: '서',
 33: '겼을',
 34: '도내',
 35: '물이',
 36: '생긴',
 37: '화악산',
 38: '어려울',
 39: '남부',
 40: '14일',
 41: '어깨',
 42: '강변',
 43: '그러한',
 44: '때',
 45: '05일',
 46: '으로',
 47: '실외',
 48: '라디오',
 49: '5월',
 50: '여유',
 51: '백운산',
 52: '반',
 53: '여주시',
 54: '검은',
 55: '나',
 56: '완만',
 57: '지날',
 58: '시청',
 59: '오후',
 60: '북동쪽',
 61: '15일',
 62: '전라남도',
 63: '야',
 64: '건넌다',
 65: '근육',
 66: '뒤쪽',
 67: '우천',
 68: '청',
 69: '낮게하고',
 70: '로프',
 71: '놀라서',
 72: '경주',
 73: '자',
 74: '거북이',
 75: '암벽',
 76: '건너지',
 77: '밀고',
 78: '끼',
 79: '달리는',
 80: '사람',
 81: '이나',
 82: '물기',
 83: '하천',
 84: '맡기고',
 85: '바랍니다',
 86: '남남서',
 87: '경기',
 8

In [9]:
#len(index_to_word) : 저장된 단어의 개수
OUTPUT_DIM = len(index_to_word)

In [10]:
OUTPUT_DIM

1078

In [11]:
N_LAYER = 2
DEC_DROPOUT = 0.5
EMB_DIM = 128

In [12]:
#torch.cuda.is_available() : GPU 사용가능하면 TRUE 아니면 False

#"cuda:0" if torch.cuda.is_available() else "cpu" : GPU사용 가능하면 "cuda:0" 리턴 아니면 cpu 리턴
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [13]:
#수어 번역할 동영상 경로
video_path = "C:/ict_sign_language_project01/data/mp4/test01.mp4"

In [14]:
#수어로 번역할 비디오 파일 경로
video = Video.from_file(video_path)
#수어로 번역할 비디오 화면에 출력
display(video)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...')

In [15]:
#동영상에서 추출한 프레임을 저장할 폴더
frame_path = "data/test_frame/"

#pathlib.Path(frame_path):프레임을 저장할 폴더의 정보를 알아낼 객체 리턴
frame_save_path = pathlib.Path(frame_path)

#frame_save_path.exists() : 프레임을 저장할 폴더가 존재하면 True, 아니면 False 리턴
if frame_save_path.exists():
    #shutil.rmtree(frame_path) : 프레임을 저장할 폴더 data/frame 폴더 삭제
    shutil.rmtree(frame_path)

#폴더 생성
os.makedirs(frame_save_path)

#동영상의 프레임 번호
frame_num = 0
#저장된 jpg의 순서
jpg_num = 0

#cv2.VideoCapture(video_path): 수어 동영상 파일의 정보를 가져올 객체 생성
capture = cv2.VideoCapture(video_path)

while(True):
    #capture.read(): 동영상의 프레임을 읽어서 리턴

    #success: 프레임을 읽어 올 수 있으면 True, 더이상 읽을 프레임이 없으면 False
    #frame: 동영상의 프레임 저장
    success, frame = capture.read()

    #(frame_num % SKIP_NUM == 0) : frame_num(동영상 프레임 순서)를 SKIP_NUM으로 나눈 나머지가 0
                                # SKIP_NUM이 3이므로 3의 배수번째 프레임

    if success & (frame_num % SKIP_NUM == 0):
        #저장할 프레임 경로 
        jpg_path =f'{frame_save_path}/frame_{jpg_num}.jpg'
        #동영상 프레임 저장
        cv2.imwrite(jpg_path, frame)
        #저장한 jpg 개수 1증가
        jpg_num = jpg_num + 1

    if not success:
        break 

    #프레임 개수 1 증가
    frame_num = frame_num+1

try:
    os.makedirs(frame_save_path)
except Exception as e:
    print(f"Error creating directory: {e}")


#동영상 닫기
capture.release()

Error creating directory: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'data\\test_frame'


In [16]:
#자세와 손의 키 포인트를 저장할 폴더 경로
keypoint_path = "data/test_keypoint/"

#pathlib.Path(keypoint_path) : keypoint_path(자세와 손의 키포인트를 저장할 폴더 경로) 정보 리턴
save_path = pathlib.Path(keypoint_path)

#save_path.exists(): keypoint_path(자세와 손의 키포인트를 저장할 폴더 경로)가 존재하면 True 존재하지 않으면 False
if save_path.exists():
    #shutil.rmtree(keypoint_path): 자세와 손의 키포인트를 저장할 폴더 경로 data/test_keypoint 폴더 삭제
    shutil.rmtree(keypoint_path)

#폴더 생성
os.makedirs(save_path)

In [17]:
#openpose\\bin\\OpenPoseDemo.exe : keypoint 생성할 파일 경로
    #--image_dir data\\test_frame: 이미지 프레임 경로 
    #--hand: 손의 좌표 키포인트도 추가
    #--model_folder openpose\\models : 모델 파일 경로
    #--write_json data \\test_keypoint : 키포인트 json 파일 저장 경로
    #--display 0 --render_pose 0 : 키포인트 결과를 파일로 저장
    #--num_gpu 1 -- num_gpu_start 0 : GPU 설정
command = "openpose\\bin\\OpenPoseDemo.exe --image_dir data\\test_frame --hand --model_folder openpose\\models --write_json data\\test_keypoint --display 0 --render_pose 0 --num_gpu 1 --num_gpu_start 0"
print("="*100)
print(command)
print("="*100)
#command실행
subprocess.run(command)

openpose\bin\OpenPoseDemo.exe --image_dir data\test_frame --hand --model_folder openpose\models --write_json data\test_keypoint --display 0 --render_pose 0 --num_gpu 1 --num_gpu_start 0


CompletedProcess(args='openpose\\bin\\OpenPoseDemo.exe --image_dir data\\test_frame --hand --model_folder openpose\\models --write_json data\\test_keypoint --display 0 --render_pose 0 --num_gpu 1 --num_gpu_start 0', returncode=0)

In [18]:
#os.listdir(keypoint_path) : keypoint_path 에 저장된 JSON 파일 리스트 리턴
keypoint_file_list = os.listdir(keypoint_path)

#len(keypoint_file_list) : 폴더에 저장된 JSON 파일의 개수 
keypoint_num = len(keypoint_file_list)

#JSON 파일의 내용을 저장할 리스트
all_keypoint_list = []

for i in range(keypoint_num):
    #JSON 파일의 경로와 파일명
    json_file_path = f"{keypoint_path}/frame_{i}_keypoints.json"

    with open(json_file_path, "r") as keypoint_json_file:
        #json.load(keypoint_json_file) : 키포인트가 저장된 JSON 파일의 내용 리턴
        keypoint_json = json.load(keypoint_json_file)

        #keypoint_json["people"][0]["pose_keypoints_2d"]: 포즈의 키포인트 리턴
        pose_keypoints_2d = keypoint_json["people"][0]["pose_keypoints_2d"]

        #keypoint_json["people"][0]["hand_left_keypoints_2d"] : 왼손의 키포인트 리턴
        hand_left_keypoints_2d = keypoint_json["people"][0]["hand_left_keypoints_2d"]

        #keypoint_json["people"][0]["hand_right_keypoint_2d"] : 오른손 키포인트 리턴
        hand_right_keypoints_2d = keypoint_json["people"][0]["hand_right_keypoints_2d"]

        #pose_keypoints_2d: 포즈의 키포인트가 저장된 리스트 각 keypoint 마다 x좌표 y좌표 확률 3개씩 저장
        #np.array(pose_keypoints_2d) : numpy 배열로 변환
        #reshape(-1, 3): 리스트를 3열로 변환 행은 생략 (-1)
        pose_keypoint_arr = np.array(pose_keypoints_2d).reshape(-1,3)

        #hand_left_keypoints_2d: 왼손의 키포인트가 저장된 리스트 각 keypoint 마다 x좌표 y좌표 확률 3개씩 저장
        #np.array(pose_keypoints_2d): numpy배열로 변환
        #reshape(-1,3): 리스트를 3열로 변환 행은 생략(-1)
        hand_left_keypoint_arr = np.array(hand_left_keypoints_2d).reshape(-1,3)

        #hand_right_keypoints_2d: 오른손 키포인트가 저장된 리스트 각 keypoint 마다 x좌표 y좌표 확률 3개씩 저장
        #np.array(hand_right_keypoints_2d) : numpy 배열로 변환
        #reshape(-1,3) : 리스트를 3열로 변환 행은 생략 (-1)

        hand_right_keypoint_arr = np.array(hand_right_keypoints_2d).reshape(-1, 3)

        #pose_keypoint_arr[:, :2] : pose_keypoint_arr (포즈 키포인트의 x좌표 y좌표 확률 저장의
                                                    #  모든 행(:) 2열 미만( : 2 ) 리턴 => x,y 좌표 리턴

        #hand_left_keypoint_arr[:, :2] : hand_left_keypoint_arr (왼손 키포인트의 x좌표 y좌표 확률 저장의 
                                            #모든행 (:) 2열 미만 (: 2 ) 리턴 => x,y좌표 리턴
        #hand_right_keypoint_arr[: , :2] : hand_right_keypoint_arr (오른손 키포인트의 x좌표 y좌표 확률 저장의 
                                            #모든행 (:) 2 열 미만 (: 2 ) 리턴 => x,y 좌표 리턴

        #np.concatenate (): 배열들을 합쳐서 새로운 배열 생성
        #pose_keypoint_arr [: , :2]: 포즈 키포인트의 x,y 좌표 
        #hand_left_keypoint_arr[: , :2] : 왼손 키포인트의 x,y 좌표
        #hand_right_keypoint_arr[: , :2] : 오른손 키포인트의 x,y 좌표 

        #np.concatenate((pose_keypoint_arr[: , :2], hand_left_keypoint_arr[: , :2], hand_right_keypoint_arr[: , :2]))
        # : 포즈 왼손 오른손 키포인트의 x, y 좌표를 합쳐서 새로운 배열 생성
        keypoint_arr = np.concatenate((pose_keypoint_arr[:, :2], hand_left_keypoint_arr[: , :2], hand_right_keypoint_arr[:, :2]))

        #keypoint_arr 을 all_keypoint_list에 추가 
        all_keypoint_list.append(keypoint_arr.tolist())

#np.array(all_keypoint_list, dtype ="float32") : all_keypoint_list를 실수("float32") 타입 배열로 변환
test_keypoint_arr = np.array(all_keypoint_list, dtype = "float32")

In [19]:
test_keypoint_arr

array([[[977.623, 339.02 ],
        [971.711, 483.224],
        [859.857, 483.208],
        ...,
        [808.919, 912.601],
        [819.674, 911.064],
        [828.124, 909.528]],

       [[977.622, 339.021],
        [971.73 , 483.25 ],
        [859.891, 483.227],
        ...,
        [808.299, 912.251],
        [818.886, 911.495],
        [826.448, 908.47 ]],

       [[977.633, 339.   ],
        [971.75 , 483.253],
        [859.904, 483.221],
        ...,
        [807.712, 911.006],
        [819.408, 911.006],
        [827.985, 908.667]],

       ...,

       [[977.62 , 338.95 ],
        [974.632, 486.138],
        [859.974, 486.088],
        ...,
        [830.121, 932.791],
        [836.263, 937.397],
        [844.708, 939.7  ]],

       [[977.639, 338.961],
        [974.621, 486.081],
        [859.971, 483.333],
        ...,
        [831.655, 932.742],
        [837.03 , 938.885],
        [846.245, 941.957]],

       [[977.639, 338.944],
        [974.613, 486.133],
        [859.999

In [20]:
test_keypoint_arr.shape

(151, 67, 2)

In [21]:
frame_keypoint_list = []

#test_keypoint_list의 1행씩 arr_row에 대입
for arr_row in test_keypoint_arr:
    #arr_row[:, 0]: arr_row의 0열 -> keypoint의 x좌표
    x = arr_row[:, 0]
    #arr_row[:, 1]: arr_row의 1열 -> keypoint의 y좌표
    y = arr_row[:, 1]

    #np.mean(x): keypoint의 x좌표 평균
    mean_x = np.mean(x)
    #np.mean(y): keypoint의 y좌표 평균
    mean_y = np.mean(y)

    #np.std(): keypoint의 x, y좌표 표준편차
    std_x = np.std(x)
    std_y = np.std(y)

    #keypoint의 x, y좌표에서 평균을 빼고 표준편차로 나눔
    #평균 => 0, 표준편차 => 1
    normal_x = (x - mean_x) / std_x
    normal_y = (y - mean_y) / std_y

    #0번째 열, 1번째 열에 normal_x, y 대입
    arr_row[:, 0] = normal_x
    arr_row[:, 1] = normal_y

    #arr_row.flatten(): arr_row(2차원배열)을 1차원 배열로 변환
    #tolist(): 배열을 리스트로 변환
    #frame_keypoint_list.append(): frame_keypoint에 추가
    frame_keypoint_list.append(arr_row.flatten().tolist())

In [22]:
#len(frame_keypoint_list)가 MAX_FRAME_NUM(240) 미만
if len(frame_keypoint_list) < MAX_FRAME_NUM:
    #np.random.choice(max, num): 0 ~ max 미만의 난수 num 개 리턴

    #np.random.choice(len(frame_keypoint_list), MAX_FRAME_NUM - len(frame_keypoint_list)):
    # len(frame_keypoint_list): frame_keypoint_list의 행의 개수 미만의 난수
    # MAX_FRAME_NUM - len(frame_keypoint_list)개 리턴
    random_choice_frame = np.random.choice(len(frame_keypoint_list), MAX_FRAME_NUM - len(frame_keypoint_list))

    #random_choice_frame에 저장된 난수를 정렬
    random_choice_frame.sort()

    #random_choice_frame에 저장된 난수를 random_idx에 대입
    for random_idx in random_choice_frame:
        #frame_keypoint_list[random_idx] : frame_keypoint에서 random_idx번째 프레임
        #frame_keypoint_list.append() : frame_keypoint_list에 추가
        # => 240행(프레임 키포인트 x, y 좌표)으로 길이를 맞춤
        frame_keypoint_list.append(frame_keypoint_list[random_idx])
else:
    #frame_keypoint_list[: MAX_FRAME_NUM]: MAX_FRAME_NUM 행(프레임 키포인트 x, y좌표) 리턴
    frame_keypoint_list = frame_keypoint_list[: MAX_FRAME_NUM]

#frame_keypoint_list[::-1]: frame_keypoint의 행을 역순으로 변환
frame_keypoint_list = frame_keypoint_list[::-1]

In [23]:
#frame_keypoint_list를 배열로 변환
X_test = np.array([frame_keypoint_list], dtype="float32")

X_test

array([[[ 0.3276395 , -1.2316422 ,  0.31877786, ...,  0.59847534,
         -0.05984475,  0.6007849 ],
        [ 0.3276395 , -1.2316422 ,  0.31877786, ...,  0.59847534,
         -0.05984475,  0.6007849 ],
        [ 0.32288808, -1.2310326 ,  0.32220817, ...,  0.60404617,
         -0.06420448,  0.60874337],
        ...,
        [ 0.35164458, -1.2245742 ,  0.33427715, ...,  0.5424522 ,
         -0.09013673,  0.53522664],
        [ 0.3525455 , -1.224533  ,  0.33514795, ...,  0.54440945,
         -0.09383021,  0.53506213],
        [ 0.3522563 , -1.2240092 ,  0.3347979 , ...,  0.54368204,
         -0.0892223 ,  0.5389356 ]]], dtype=float32)

In [24]:
X_test.shape

(1, 240, 134)

In [25]:
#인코더 연산을 클래스로 선언
class Encoder(nn.Module):
    def __init__(self, input_size, hid_dim, n_layer):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layer = n_layer
        self.gru = nn.GRU(
                            input_size,
                            hid_dim,
                            n_layer,
                            batch_first = True,
                            bidirectional = True
        )
        self.fc = nn.Linear(
                            hid_dim * 4,
                            hid_dim
        )

    def forward(self, x):
        h0 = torch.zeros(self.n_layer * 2, x.shape[0], self.hid_dim).cuda().float()
        encoder_output, encoder_hidden = self.gru(x, h0)

        encoder_hidden = torch.cat((
                                    encoder_hidden[-4,:,:],
                                    encoder_hidden[-3,:,:],
                                    encoder_hidden[-2,:,:],
                                    encoder_hidden[-1,:,:]
        ), dim = 1)

        encoder_hidden = torch.tanh(self.fc(encoder_hidden))
        return encoder_output, encoder_hidden

In [26]:
#Attention을 클래스로 구현

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()

        self.attn = nn.Linear((hid_dim * 2) + hid_dim, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias = False)

    def forward(self, hidden, encoder_output):
        batch_size = encoder_output.shape[0]
        src_len = encoder_output.shape[1]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_output), dim = 2)))

        attention = self.v(energy).squeeze(2)
        return F.softmax(attention, dim=1)

In [27]:
#디코더 연산을 클래스로 구현
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layer, attention, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.attention = attention
        self.n_layer = n_layer
        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.gru = nn.GRU(hid_dim * 2 + emb_dim, hid_dim)
        self.fc_out = nn.Linear((hid_dim * 2) + hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, encoder_hidden, encoder_output):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))

        attention_output = self.attention(encoder_hidden, encoder_output)
        attention_output = attention_output.unsqueeze(1)

        weighted = torch.bmm(attention_output, encoder_output)

        weighted = weighted.permute(1, 0, 2)

        rnn_input = torch.cat((embedded, weighted), dim = 2)

        decoder_output, decoder_hidden = self.gru(rnn_input, encoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        decoder_output = decoder_output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((decoder_output, weighted, embedded), dim = 1))

        return prediction, decoder_hidden.squeeze(0)

In [28]:
#SequenceToSequence 구현
class GRU_AT_Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    #딥러닝 학습 시 호출되는 함수
    #src : 수어 keypoint
    #trg : 수어 스크립트 문장이 저장
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #trg.shape[0] : trg에 저장된 문장의 개수(16)
        batch_size = trg.shape[0]

        #trg.shape[1] : 수어 스크립트 단어의 개수
        trg_len = trg.shape[1]

        #decoder.output_dim : 수어 스크립트를 구성하는 단어의 개수
        trg_vocab_size = self.decoder.output_dim

        #torch.zeros(trg_len, batch_size, trg_vocab_size) : 0으로 초기화된 배열 새엇ㅇ
                                                        #   배열 shape[trg_len(30), batch_size(16), trg_vocab_size(162)]
        output = torch.zeros(trg_len, batch_size, trg_vocab_size).cuda()

        #self.encoder(src) : 인코더 연산 실행
        encoder_output, encoder_hidden = self.encoder(src)

        #trg[:, 0] : 수어스크립트 trg 0번째 칸 리턴
        input = trg[:, 0]

        for t in range(1, trg_len):
            #self.decoder(input, encoder_hidden, encoder_output): 디코더 연산 실행
            decoder_output, decoder_hidden = self.decoder(input, encoder_hidden, encoder_output)

            output[t] = decoder_output # decoder_output: 디코더 연산 결과

            #random.random() < teacher_forcing_ratio : 0~1 사이 난수를 생성해서 난수가 0.5미만이면 난수, 이상이면 None 리턴
            teacher_force = random.random() < teacher_forcing_ratio

            #decoder_output.argmax(1) : decoder_output에서 가장 확률이 높은 단어 대입
            top1 = decoder_output.argmax(1)

            #trg[:, t] if teacher_force : techar_force가 None이 아니면 수어스크립트가 저장된 trg[:, t] 리턴
            #    else top1 : teacher_force가 None이면 decoder의 예측값 top1 리턴
            input = trg[:, t] if teacher_force else top1

        return output

In [29]:
#X_test.shape[-1] : keypoint 배열의 열 리턴
input_size = X_test.shape[-1]
input_size

134

In [30]:
enc = Encoder(input_size, HID_DIM, N_LAYER)
enc

Encoder(
  (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=2048, out_features=512, bias=True)
)

In [31]:
att = Attention(HID_DIM)
att

Attention(
  (attn): Linear(in_features=1536, out_features=512, bias=True)
  (v): Linear(in_features=512, out_features=1, bias=False)
)

In [32]:
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYER, att, DEC_DROPOUT)
dec

Decoder(
  (attention): Attention(
    (attn): Linear(in_features=1536, out_features=512, bias=True)
    (v): Linear(in_features=512, out_features=1, bias=False)
  )
  (embedding): Embedding(1078, 128)
  (gru): GRU(1152, 512)
  (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [33]:
model = GRU_AT_Seq2Seq(enc, dec, device).to(device)
model

GRU_AT_Seq2Seq(
  (encoder): Encoder(
    (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=2048, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1078, 128)
    (gru): GRU(1152, 512)
    (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [34]:
# -0.08 ~ 0.08 사이 난수 리턴
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [35]:
#-0.08 ~ 0.08 사이 난수로 초기화
model.apply(init_weights)

GRU_AT_Seq2Seq(
  (encoder): Encoder(
    (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=2048, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1078, 128)
    (gru): GRU(1152, 512)
    (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [36]:
#학습 결과가 저장된 data/model1.pt를 읽어서 model에 저장
model.load_state_dict(torch.load("data/model1.pt"))

<All keys matched successfully>

In [37]:
model

GRU_AT_Seq2Seq(
  (encoder): Encoder(
    (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=2048, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1078, 128)
    (gru): GRU(1152, 512)
    (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [38]:
#수어 keypoint 가 X_testㄹ,수어 keypoint 가 X_test를 tensor 객체로 변환
X_test = torch.tensor(X_test)
X_test

tensor([[[ 0.3276, -1.2316,  0.3188,  ...,  0.5985, -0.0598,  0.6008],
         [ 0.3276, -1.2316,  0.3188,  ...,  0.5985, -0.0598,  0.6008],
         [ 0.3229, -1.2310,  0.3222,  ...,  0.6040, -0.0642,  0.6087],
         ...,
         [ 0.3516, -1.2246,  0.3343,  ...,  0.5425, -0.0901,  0.5352],
         [ 0.3525, -1.2245,  0.3351,  ...,  0.5444, -0.0938,  0.5351],
         [ 0.3523, -1.2240,  0.3348,  ...,  0.5437, -0.0892,  0.5389]]])

In [39]:
#model을 GPU에 저장
model.cuda()

#X_test.cuda() : X_test을 GPU에 저장
#float() : 실수 타입으로 변환
X_test = X_test.cuda().float()

#model을 실행(평가) 모드로 변환 => 학습 안함
model.eval()

#torch.no_grad() : model의 값을 수정하지 않음
with torch.no_grad():
    #model.encoder(X_test): 인코더 실행
    encoder_outputs, hidden = model.encoder(X_test)

sentence_list = []
last_word = None

for index in range(30):
    #Decoder의 첫번째 입력 [STA_INDEX] 를 tensor 객체로 변환
    start_tensor = torch.tensor([STA_INDEX], dtype = torch.long).to(device)

    #torch.no_grad() : model의 값을 수정하지 않음
    with torch.no_grad():
        #Decoder 실행
        decoder_output, hidden = model.decoder(start_tensor, hidden, encoder_outputs)

    #decoder_output : Decoder의 예측결과
    #decoder_output.argmax(1) : Decoder의 예측결과에서 가장 확률이 높은 단어 인덱스 리턴 => 2차원배열}
    #.item() : 배열에 저장된 데이터 리턴
    pred_token = decoder_output.argmax(1).item()
    print("pred_token=", pred_token)
    #index_to_word.get(pred_token) : 예측된 단어 인덱스를 단어로 변환
    print("pred_word=", index_to_word.get(pred_token))
    print("=" * 100)

    if pred_token == END_INDEX:
        break

    if last_word != index_to_word.get(pred_token):
        #예측된 단어를 추가
        #index_to_word.get(pred_token) : 예측된 단어 인덱스를 단어로 변환
        sentence_list.append(index_to_word.get(pred_token))
        
    last_word = index_to_word.get(pred_token)

pred_token= 505
pred_word= 경북
pred_token= 499
pred_word= 포항
pred_token= 226
pred_word= 규모
pred_token= 916
pred_word= 5
pred_token= 916
pred_word= 5
pred_token= 916
pred_word= 5
pred_token= 847
pred_word= 지진
pred_token= 847
pred_word= 지진
pred_token= 847
pred_word= 지진
pred_token= 900
pred_word= 발생
pred_token= 662
pred_word= 여진
pred_token= 757
pred_word= 등
pred_token= 123
pred_word= 안전
pred_token= 840
pred_word= 주의
pred_token= 85
pred_word= 바랍니다
pred_token= 85
pred_word= 바랍니다
pred_token= 2
pred_word= <END>


In [40]:
sentence_list

['경북', '포항', '규모', '5', '지진', '발생', '여진', '등', '안전', '주의', '바랍니다']

In [41]:
#sentence_list를 문자열로 변환
" ".join(sentence_list)

'경북 포항 규모 5 지진 발생 여진 등 안전 주의 바랍니다'